# Variational Deep Q Network

paper accepted at NIPS 2017 Bayesian Deep Learning workshop.


### Markov Decision Process (MDP) and Reinforcement Learning (RL)

In Markov Decision Process (MDP), an agent is in state $s_t \in \mathcal{S}$, takes action $a_t \in \mathcal{A}$, then transitions to state $s_{t+1} \in \mathcal{S}$ and receives instant reward $r_t$. A mapping from state to action $\pi: \mathcal{S} \mapsto \mathcal{A}$ is a policy. The aim is to find policy to optimize cumulative reward
$$\mathbb{E}_\pi\big[ \sum_{t=0}^\infty r_t\gamma^t  \big]$$
where $\gamma$ is a discount factor. 

#### Action value function $Q^\pi(s,a)$
An agent starts with state action pair $s_0 = s,a_0 = a$ then follows policy $\pi$, then the action value function is defined as $$Q^\pi(s,a) = \mathbb{E}_\pi \big[ \sum_{t=0}^\infty r_t \gamma^t \big| s_0 = s, a_0 = a \big]$$

#### Bellman quation of optimality
The optimal policy $\pi^\ast$ has optimal action value function $Q^\ast(s,a)$ satisfy the following equality $$Q^\ast(s_t,a_t) = \max_a \mathbb{E}\big[ r_t + \gamma Q^\ast(s_{t+1},a) \big]$$
To obtain $\pi^\ast$ from $Q^\ast(s,a)$: $\pi^\ast(s) = \arg\max_a Q^\ast(s,a)$ (one-step lookahead).

#### Bellman error
One way to evaluate how suboptimal a policy is, is through Bellman error. Bellman error for state action pair $(s,a)$ under policy $\pi$ is $$(Q^\pi(s_t,a_t) - \max_a \mathbb{E}\big[ r_t + \gamma Q^\pi(s_{t+1},a) \big])^2$$

Notice that optimal $Q^\ast(s,a)$ has exact zero Bellman error for all $(s,a)$. A policy $\pi$ with zero Bellman error is also optimal. Suboptimal policies tend to have small Bellman error (though not necessarily).


### Deep Q Network (DQN)
Deep Q Network is proposed in [Mnih, 2013], as one of the first successful deep RL framework to solve challenging control tasks. Consider parameterizing a neural network $Q_\theta(s,a)$ with parameter $\theta$ to approximate $Q^\ast(s,a)$. Use SGD to minimize the objective
$$\mathbb{E}_{\pi_\theta} \big[(Q_\theta(s_t,a_t) - \max_a \mathbb{E}\big[ r_t + \gamma Q_\theta(s_{t+1},a) \big])^2\big]$$

If $Q_\theta(s,a)$ minimizes the above objective then potentially $Q_\theta(s,a) \approx Q^\ast(s,a)$ and greedy policy w.r.t. $Q_\theta(s,a)$ is close to optimal.

### Exploration in RL
Exploration is essential in solving RL tasks. Being greedy usually gives suboptimal performance because the agent does not explore and cannot find potentially more optimal patterns. Efficient exploration scheme is critical in solving challenging control tasks.

### Variational Deep Q Network (VDQN)
Original DQN uses $\epsilon-$greedy exploration. Such exploration scheme is myopic and is not efficient enough in challenging tasks. Variational DQN maintains a distribution over DQN parameters $\theta \sim q_\phi(\theta)$ with parameter $\phi$.

Proposed objective: $$\mathbb{E}_{\theta\sim q_\phi(\theta)} \big[ \mathbb{E}_{\pi_\theta} \big[(Q_\theta(s_t,a_t) - \max_a \mathbb{E}\big[ r_t + \gamma Q_\theta(s_{t+1},a) \big])^2\big]  \big] - \lambda \mathbb{H}(q_\phi(\theta))$$

First term encourages minimization of expected Bellman error -- search for optimal policy; Second term encourages high entropy of distribution $q_\phi(\theta)$ -- encourage exploration. 

In practice, the 'regression target' $d_t = \max_a \mathbb{E}\big[ r_t + \gamma Q_\theta(s_{t+1},a)\big]$ is computed by a target network. Let targets $D = \{d_t\}$ be data. We can interpret $Q_\theta(s,a)$ as a Bayesian neural network with uniform improper prior $p(\theta) \propto 1$, the above objective reduces to 
$$\mathbb{KL}\big[q_\phi(\theta) \ || \ p(\theta|D)\big]$$
we can use Edward to update distribution parameter $\phi$.


In [1]:
import edward
import numpy as np
import tensorflow as tf
import gym

## VDQN architecture

For control tasks in current experiments, we use feed forward neural network. The input is state $s_t$ at each time step. The feed forward network has 2 hidden layers each with $100$ hidden units with relu activation. The output is a probability vector $p$ of choosing actions (discrete action space).

In [3]:
ed.set_seed(42)

data = np.loadtxt('data/crabs_train.txt', delimiter=',')
data[data[:, 0] == -1, 0] = 0  # replace -1 label with 0 label

N = data.shape[0]  # number of data points
D = data.shape[1] - 1  # number of features

X_train = data[:, 1:]
y_train = data[:, 0]

print("Number of data points: {}".format(N))
print("Number of features: {}".format(D))

Number of data points: 100
Number of features: 5


## Model

A Gaussian process is a powerful object for modeling nonlinear
relationships between pairs of random variables. It defines a distribution over
(possibly nonlinear) functions, which can be applied for representing
our uncertainty around the true functional relationship.
Here we define a Gaussian process model for classification
(Rasumussen & Williams, 2006).

Formally, a distribution over functions $f:\mathbb{R}^D\to\mathbb{R}$ can be specified
by a Gaussian process
$$
\begin{align*}
  p(f)
  &=
  \mathcal{GP}(f\mid \mathbf{0}, k(\mathbf{x}, \mathbf{x}^\prime)),
\end{align*}
$$
whose mean function is the zero function, and whose covariance
function is some kernel which describes dependence between
any set of inputs to the function.

Given a set of input-output pairs
$\{\mathbf{x}_n\in\mathbb{R}^D,y_n\in\mathbb{R}\}$,
the likelihood can be written as a multivariate normal

\begin{align*}
  p(\mathbf{y})
  &=
  \text{Normal}(\mathbf{y} \mid \mathbf{0}, \mathbf{K})
\end{align*}

where $\mathbf{K}$ is a covariance matrix given by evaluating
$k(\mathbf{x}_n, \mathbf{x}_m)$ for each pair of inputs in the data
set.

The above applies directly for regression where $\mathbb{y}$ is a
real-valued response, but not for (binary) classification, where $\mathbb{y}$
is a label in $\{0,1\}$. To deal with classification, we interpret the
response as latent variables which is squashed into $[0,1]$. We then
draw from a Bernoulli to determine the label, with probability given
by the squashed value.

Define the likelihood of an observation $(\mathbf{x}_n, y_n)$ as

\begin{align*}
  p(y_n \mid \mathbf{z}, x_n)
  &=
  \text{Bernoulli}(y_n \mid \text{logit}^{-1}(\mathbf{x}_n^\top \mathbf{z})).
\end{align*}

Define the prior to be a multivariate normal

\begin{align*}
  p(\mathbf{z})
  &=
  \text{Normal}(\mathbf{z} \mid \mathbf{0}, \mathbf{K}),
\end{align*}

with covariance matrix given as previously stated.

Let's build the model in Edward. We use a radial basis function (RBF)
kernel, also known as the squared exponential or exponentiated
quadratic. It returns the kernel matrix evaluated over all pairs of
data points; we then Cholesky decompose the matrix to parameterize the
multivariate normal distribution.

In [4]:
X = tf.placeholder(tf.float32, [N, D])
f = MultivariateNormalTriL(loc=tf.zeros(N), scale_tril=tf.cholesky(rbf(X)))
y = Bernoulli(logits=f)

Here, we define a placeholder `X`. During inference, we pass in
the value for this placeholder according to data.

## Inference

Perform variational inference.
Define the variational model to be a fully factorized normal.

In [5]:
qf = Normal(loc=tf.Variable(tf.random_normal([N])),
            scale=tf.nn.softplus(tf.Variable(tf.random_normal([N]))))

Run variational inference for `500` iterations.

In [6]:
inference = ed.KLqp({f: qf}, data={X: X_train, y: y_train})
inference.run(n_iter=5000)

5000/5000 [100%] ██████████████████████████████ Elapsed: 9s | Loss: 82.755


In this case
`KLqp` defaults to minimizing the
$\text{KL}(q\|p)$ divergence measure using the reparameterization
gradient.
For more details on inference, see the [$\text{KL}(q\|p)$ tutorial](/tutorials/klqp).
(This example happens to be slow because evaluating and inverting full
covariances in Gaussian processes happens to be slow.)